# Reto:Deserción de Empleados

### 1. Librerías

In [1]:
import piplite
await piplite.install(["pandas", "numpy", "scikit-learn"])
%pip install -q scikit-learn

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

### 2. Lee el archivo CSV llamado empleadosRETO.csv y coloca los datos en un frame de Pandas llamado EmpleadosAttrition.

In [3]:
EmpleadosAttrition = pd.read_csv("data/empleadosRETO.csv")
EmpleadosAttrition.head()

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,...,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsInCurrentRole,YearsSinceLastPromotion,Attrition
0,50,Travel_Rarely,Research & Development,1 km,2,Medical,1,997,4,Male,...,22,4,3,80,32,1,2,4,1,No
1,36,Travel_Rarely,Research & Development,6 km,2,Medical,1,178,2,Male,...,20,4,4,80,7,0,3,2,0,No
2,21,Travel_Rarely,Sales,7 km,1,Marketing,1,1780,2,Male,...,13,3,2,80,1,3,3,0,1,Yes
3,52,Travel_Rarely,Research & Development,7 km,4,Life Sciences,1,1118,2,Male,...,19,3,4,80,18,4,3,6,4,No
4,33,Travel_Rarely,Research & Development,15 km,1,Medical,1,582,2,Male,...,12,3,4,80,15,2,4,6,7,Yes


### 3. Elimina las columnas que, con alta probabilidad (estimada por ti), no tienen relación alguna con la salida. Hay algunas columnas que contienen información que no ayuda a definir el desgaste de un empleado, tal es caso de las siguientes:

In [4]:
EmpleadosAttrition = EmpleadosAttrition.drop(columns=["EmployeeCount", "EmployeeNumber", "Over18", "StandardHours"])
EmpleadosAttrition.head()

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,...,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsInCurrentRole,YearsSinceLastPromotion,Attrition
0,50,Travel_Rarely,Research & Development,1 km,2,Medical,4,Male,3,4,...,No,22,4,3,32,1,2,4,1,No
1,36,Travel_Rarely,Research & Development,6 km,2,Medical,2,Male,3,2,...,No,20,4,4,7,0,3,2,0,No
2,21,Travel_Rarely,Sales,7 km,1,Marketing,2,Male,3,1,...,No,13,3,2,1,3,3,0,1,Yes
3,52,Travel_Rarely,Research & Development,7 km,4,Life Sciences,2,Male,3,3,...,No,19,3,4,18,4,3,6,4,No
4,33,Travel_Rarely,Research & Development,15 km,1,Medical,2,Male,3,3,...,Yes,12,3,4,15,2,4,6,7,Yes


### 4. Analiza la información proporcionada, si detectaste que no se cuenta con los años que el empelado lleva en la compañía y parece ser un buen dato. Dicha cantidad se puede calcular con la fecha de contratación ‘HiringDate’.

In [5]:
hd = EmpleadosAttrition["HiringDate"].astype(str).str.strip()
hd_dt = pd.to_datetime(hd, errors="coerce") 

hd[hd_dt.isna()].value_counts()   #arroja las fechas que estén mal

HiringDate
2/30/2012    1
Name: count, dtype: int64

#### a.	Crea una columna llamada Year y obtén el año de contratación del empleado a partir de su fecha ‘HiringDate’. No se te olvide que debe ser un entero.

In [6]:
EmpleadosAttrition["Year"] = hd.str.extract(r"(\d{4})$")[0].astype("Int64")

#### b.	Crea una columna llamada YearsAtCompany que contenga los años que el empleado lleva en la compañía hasta el año 2018. Para su cálculo, usa la variable Year que acabas de crear.


In [7]:
EmpleadosAttrition["YearsAtCompany"] = (2018 - EmpleadosAttrition["Year"]).astype("Int64")
EmpleadosAttrition[["HiringDate", "Year", "YearsAtCompany"]].head(10)

,HiringDate,Year,YearsAtCompany
0,06/06/2013,2013,5
1,12/25/2015,2015,3
2,2/14/2017,2017,1
3,7/29/2010,2010,8
4,10/07/2011,2011,7
5,10/17/1996,1996,22
6,1/16/2016,2016,2
7,11/28/2012,2012,6
8,8/25/2006,2006,12
9,1/21/2012,2012,6


In [8]:
EmpleadosAttrition[EmpleadosAttrition["Year"].isna()][["HiringDate"]].head(20) #Identifica si hay nulos o errores

,HiringDate


### 5. La DistanceFromHome está dada en kilómetros, pero tiene las letras “km” al final y así no puede ser entera.

#### a. Renombra la variable DistanceFromHome a DistanceFromHome_km.

In [9]:
EmpleadosAttrition = EmpleadosAttrition.rename(columns={"DistanceFromHome": "DistanceFromHome_km"})

#### b. Crea una nueva variable DistanceFromHome que sea entera, es decir, solo con números.


In [10]:
EmpleadosAttrition["DistanceFromHome"] = (
    EmpleadosAttrition["DistanceFromHome_km"]
    .astype(str)
    .str.lower()
    .str.replace("km", "", regex=False)
    .str.strip()
    .str.replace(r"[^0-9\-]", "", regex=True)
)

EmpleadosAttrition["DistanceFromHome"] = pd.to_numeric(EmpleadosAttrition["DistanceFromHome"]).astype("Int64")

EmpleadosAttrition[["DistanceFromHome_km", "DistanceFromHome"]].head()

,DistanceFromHome_km,DistanceFromHome
0,1 km,1
1,6 km,6
2,7 km,7
3,7 km,7
4,15 km,15


### 6. Borra las columnas Year, HiringDate y DistanceFromHome_km debido a que ya no son útiles.

In [11]:
EmpleadosAttrition = EmpleadosAttrition.drop(
    columns=["Year", "HiringDate", "DistanceFromHome_km"],
    errors="ignore"
)

EmpleadosAttrition.head()

,Age,BusinessTravel,Department,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,...,PerformanceRating,RelationshipSatisfaction,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsInCurrentRole,YearsSinceLastPromotion,Attrition,YearsAtCompany,DistanceFromHome
0,50,Travel_Rarely,Research & Development,2,Medical,4,Male,3,4,Research Director,...,4,3,32,1,2,4,1,No,5,1
1,36,Travel_Rarely,Research & Development,2,Medical,2,Male,3,2,Manufacturing Director,...,4,4,7,0,3,2,0,No,3,6
2,21,Travel_Rarely,Sales,1,Marketing,2,Male,3,1,Sales Representative,...,3,2,1,3,3,0,1,Yes,1,7
3,52,Travel_Rarely,Research & Development,4,Life Sciences,2,Male,3,3,Healthcare Representative,...,3,4,18,4,3,6,4,No,8,7
4,33,Travel_Rarely,Research & Development,1,Medical,2,Male,3,3,Manager,...,3,4,15,2,4,6,7,Yes,7,15


### 7. Aprovechando los ajustes que se están haciendo, la empresa desea saber si todos los departamentos tienen un ingreso promedio similar. Genera una nuevo frame llamado SueldoPromedioDepto que contenga el MonthlyIncome promedio por departamento de los empleados y colócalo en una variable llamada SueldoPromedio. Esta tabla solo es informativa, no la vas a utilizar en el set de datos que estás construyendo.

In [12]:
SueldoPromedioDepto = (
    EmpleadosAttrition
    .groupby("Department", as_index=False)["MonthlyIncome"]
    .mean()
    .rename(columns={"MonthlyIncome": "SueldoPromedio"})
)

In [13]:
SueldoPromedioDepto

,Department,SueldoPromedio
0,Human Resources,6239.888889
1,Research & Development,6804.149813
2,Sales,7188.250000


### 8. La variable MonthlyIncome tiene un valor numérico muy grande comparada con las otras variables. Escala dicha variable para que tenga un valor entre 0 y 1. 

In [14]:
min_val = EmpleadosAttrition["MonthlyIncome"].min()
max_val = EmpleadosAttrition["MonthlyIncome"].max()

EmpleadosAttrition["MonthlyIncome"] = (EmpleadosAttrition["MonthlyIncome"] - min_val) / (max_val - min_val)

EmpleadosAttrition["MonthlyIncome"].describe()

count    400.000000
mean       0.311196
std        0.258308
min        0.000000
25%        0.106939
50%        0.222711
75%        0.462745
max        1.000000
Name: MonthlyIncome, dtype: float64

### 9. Todo parece indicar que las variables categóricas que quedan sí son importantes para obtener la variable de salida. Convierte todas las variables categóricas que quedan a numéricas:

In [15]:
EmpleadosAttrition["Attrition"] = (
    EmpleadosAttrition["Attrition"]
    .astype(str)
    .str.strip()
    .str.lower()
    .map({"yes": 1, "no": 0})
)

EmpleadosAttrition["Attrition"].value_counts(dropna=False)

Attrition
0    334
1     66
Name: count, dtype: int64

In [16]:
#Variables Categóricas
categoricas = [
    "BusinessTravel",
    "Department",
    "EducationField",
    "Gender",
    "JobRole",
    "MaritalStatus"
]

EmpleadosAttrition = pd.get_dummies(
    EmpleadosAttrition,
    columns=categoricas,
    drop_first=True,
    dtype=int
)

In [17]:
EmpleadosAttrition.head()

,Age,Education,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,...,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single
0,50,2,4,3,4,4,0.864269,9,No,22,...,0,0,0,0,1,0,0,0,0,0
1,36,2,2,3,2,2,0.207340,6,No,20,...,0,0,0,1,0,0,0,0,0,0
2,21,1,2,3,1,2,0.088062,1,No,13,...,0,0,0,0,0,0,0,1,0,1
3,52,4,2,3,3,2,0.497574,7,No,19,...,0,0,0,0,0,0,0,0,0,1
4,33,1,2,3,3,3,0.664470,7,Yes,12,...,0,0,1,0,0,0,0,0,1,0


### 10.	Ahora debes hacer la evaluación de las variables para quedarte con las mejores. Calcula la correlación lineal de cada una de las variables con respecto al Attrition.


In [18]:
corr = EmpleadosAttrition.corr(numeric_only=True)["Attrition"].drop("Attrition")

# Tabla ordenada 
CorrAttrition = pd.DataFrame({
    "corr": corr,
    "abs_corr": corr.abs()
}).sort_values("abs_corr", ascending=False)

CorrAttrition

,corr,abs_corr
JobLevel,-0.214266,0.214266
TotalWorkingYears,-0.213329,0.213329
Age,-0.212121,0.212121
MaritalStatus_Single,0.205849,0.205849
YearsInCurrentRole,-0.203918,0.203918
MonthlyIncome,-0.194936,0.194936
JobRole_Sales Representative,0.191294,0.191294
YearsAtCompany,-0.176001,0.176001
JobInvolvement,-0.166785,0.166785
JobSatisfaction,-0.164957,0.164957


### 11.	Selecciona solo aquellas variables que tengan una correlación mayor o igual a 0.1, dejándolas en otro frame llamado EmpleadosAttritionFinal. No olvides mantener la variable de salida Attrition; esto es equivalente a borrar las que no cumplen con el límite.

In [19]:
mejores_vars = CorrAttrition[CorrAttrition["abs_corr"] >= 0.1].index.tolist()

EmpleadosAttritionFinal = EmpleadosAttrition[mejores_vars + ["Attrition"]].copy()

print("Variables seleccionadas:", len(mejores_vars))
print("Shape EmpleadosAttritionFinal:", EmpleadosAttritionFinal.shape)

EmpleadosAttritionFinal.head()

Variables seleccionadas: 14
Shape EmpleadosAttritionFinal: (400, 15)


,JobLevel,TotalWorkingYears,Age,MaritalStatus_Single,YearsInCurrentRole,MonthlyIncome,JobRole_Sales Representative,YearsAtCompany,JobInvolvement,JobSatisfaction,EducationField_Technical Degree,JobRole_Laboratory Technician,EnvironmentSatisfaction,JobRole_Research Director,Attrition
0,4,32,50,0,4,0.864269,0,5,3,4,0,0,4,1,0
1,2,7,36,0,2,0.207340,0,3,3,2,0,0,2,0,0
2,1,1,21,1,0,0.088062,1,1,3,2,0,0,2,0,1
3,3,18,52,1,6,0.497574,0,8,3,2,0,0,2,0,0
4,3,15,33,0,6,0.664470,0,7,3,3,0,0,2,0,1


### 12.	Crea una nueva variable llamada EmpleadosAttritionPCA formada por los componentes principales del frame EmpleadosAttritionFinal. Recuerda que el resultado del proceso PCA es un numpy array, por lo que, para hacer referencia a una columna, por ejemplo, la 0, puedes usar la instrucción EmpleadosAttritionPCA[:,0]).

In [20]:
df = EmpleadosAttritionFinal.drop(columns=["Attrition"]).copy()


# Asegurar numérico e imputar faltantes con mediana
X = df.apply(pd.to_numeric, errors="coerce")
X = df.fillna(X.median(numeric_only=True))

# Estandarizar (media 0, desviación estándar 1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA (genera todos los componentes)
pca = PCA()
EmpleadosAttritionPCA = pca.fit_transform(X_scaled) 

# Verifica tamaño
EmpleadosAttritionPCA.shape

(400, 14)

### 13.	Agrega el mínimo número de Componentes Principales en columnas del frame EmpleadosAttritionPCA que logren explicar el 80% de la varianza, al frame EmpleadosAttritionFinal. Puedes usar la instrucción assign, columna por columna, llamando a cada una C0, C1, etc., hasta las que vayas a agregar.

In [21]:
var_acum = np.cumsum(pca.explained_variance_ratio_)

# k mínimo para llegar a 80%
k = np.argmax(var_acum >= 0.80) + 1

print("k =", k)
print("Varianza acumulada con k =", var_acum[k-1])

for i in range(k):
    EmpleadosAttritionFinal = EmpleadosAttritionFinal.assign(**{f"C{i}": EmpleadosAttritionPCA[:, i]})

EmpleadosAttritionFinal.head()

k = 8
Varianza acumulada con k = 0.8321985154428057


,JobLevel,TotalWorkingYears,Age,MaritalStatus_Single,YearsInCurrentRole,MonthlyIncome,JobRole_Sales Representative,YearsAtCompany,JobInvolvement,JobSatisfaction,...,JobRole_Research Director,Attrition,C0,C1,C2,C3,C4,C5,C6,C7
0,4,32,50,0,4,0.864269,0,5,3,4,...,1,0,4.001983,-1.539606,-1.188405,0.658541,-0.222680,1.221037,0.475621,2.010759
1,2,7,36,0,2,0.207340,0,3,3,2,...,0,0,-0.952698,-0.429730,-0.428616,-0.272986,-0.882131,-0.054493,0.086939,0.017333
2,1,1,21,1,0,0.088062,1,1,3,2,...,0,1,-3.444215,-2.326411,2.509489,-1.503448,0.286083,0.361601,-0.781089,-0.144886
3,3,18,52,1,6,0.497574,0,8,3,2,...,0,0,1.516849,-1.180583,-0.210180,0.004336,0.097543,-0.218708,-0.763480,-1.677063
4,3,15,33,0,6,0.664470,0,7,3,3,...,0,1,1.130935,0.042596,0.136315,0.068764,-0.536846,0.242105,-0.445116,0.292685


### 14.	Guarda el set de datos que has formado y que tienes en EmpleadosAttritionFinal en un archivo CSV llamado EmpleadosAttritionFinal.csv. Las últimas columnas que colocaste quedarán después de la variable Attrition, lo cual no importa, pero si gustas lo puedes arreglar antes de escribir el archivo.


In [22]:
EmpleadosAttritionFinal.to_csv("EmpleadosAttritionFinal.csv", index=False)

### 15.	Descarga tu script (archivo con extensión .ipynb) y guárdalo en un archivo que siga la nomenclatura que se te indica en Formato de entrega de actividad.